In [13]:
import torch
import torch.nn as nn
import clip
from PIL import Image
import pandas as pd
import requests
import os.path as osp
import os
import pickle
import random
import numpy as np
from pathlib import Path
import sys
from rembg import remove
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
import numpy as np

In [16]:
def remove_bg(img_path, out_path):
    input_img = Image.open(img_path)
    output_img = remove(input_img)
    output_img.save(out_path)

In [17]:
index = ["color", "color_name", "hex", "R", "G", "B"]
csv = pd.read_csv('colors.csv', names=index, header=None)

def get_color_name(r, g, b):
    
    R = r * 255
    G = g * 255
    B = b * 255
    minimum = 10000
    
    for i in range(len(csv)):
        d = (
            abs(R- int(csv.loc[i,"R"])) + 
            abs(G- int(csv.loc[i,"G"])) + 
            abs(B- int(csv.loc[i,"B"]))
        )
        
        if(d<=minimum):
            minimum = d
            cname = csv.loc[i,"color_name"]

    return cname

def get_color_names(colors):
    return set([get_color_name(c[0], c[1], c[2]) for c in colors])

In [18]:
def get_top_colors(img_path, n=3):
    image = plt.imread(img_path)
    
    n_clusters = 9
    rows, cols, channels = image.shape  # get the image shape
    X = image.reshape(rows * cols, channels)
    kmeans = KMeans(n_clusters=n_clusters, random_state=0).fit(X)
    labels = kmeans.labels_.reshape(rows, cols)
    centers = kmeans.cluster_centers_.clip(0, 1)
    
    unique_labels, counts = np.unique(labels, return_counts=True)
    res = tuple(zip(centers[unique_labels], counts))
    
    colors = []
    counts = []
    for r in res:
        if r[0][3] < 0.2:
            # background no alpha
            continue
        colors.append(r[0])
        counts.append(r[1])
    
    color_order = np.argsort(counts)[::-1]
    colors = np.array(colors)[color_order][:n]
    counts = np.array(counts)[color_order][:n]
    return colors, counts

In [23]:
def to_png(image):
    file_name = image.split(".")[0] + ".png"
    return file_name

In [14]:
image_storage = "C:/Users/aphri/Documents/t0002/work/data/w210_data/target_images"
Path(image_storage).mkdir(parents=True, exist_ok=True)

image_nobg = "C:/Users/aphri/Documents/t0002/work/data/w210_data/target_images_nobg"
Path(image_nobg).mkdir(parents=True, exist_ok=True)

In [8]:
image_files = os.listdir(image_storage)

In [15]:
total_count = len(image_files)
print_increment = total_count/10

In [ ]:
count = 0
print_count = 0
for image in image_files:
    image_path = osp.join(image_storage, image)
    out_path = osp.join(image_nobg, to_png(image))
    if not osp.exists(out_path):
        remove_bg(image_path, out_path)
    
    count += 1
    if count >= print_count:
        print(f"{round(print_count*100/total_count, 2)}%")
        print_count += print_increment
        print_count = min(print_count, total_count)

0.0%
10.0%
